## Notebook for formatting data frames for phase association
Source:
1. https://chatgpt.com/


In [1]:
import pandas as pd
import numpy as np
import datetime
import glob
import os 

### 1.1 Load the station data frame

In [2]:
region = 'swa_shore'
stas_2011 = pd.read_csv(f'../data/datasets_2011/stas_2011_{region}.csv',index_col=0)
stas_2012 = pd.read_csv(f'../data/datasets_2012/stas_2012_{region}.csv',index_col=0)
stas_2013 = pd.read_csv(f'../data/datasets_2013/stas_2013_{region}.csv',index_col=0)
stas_2014 = pd.read_csv(f'../data/datasets_2014/stas_2014_{region}.csv',index_col=0)
stas_2015 = pd.read_csv(f'../data/datasets_2015/stas_2015_{region}.csv',index_col=0)
print(stas_2011)
print(stas_2012)
print(stas_2013)
print(stas_2014)
print(stas_2015)

         id   longitude   latitude  elevation
0  7D.J49A. -124.427498  46.437801     -120.0
1  UW.MEGW. -123.877090  46.266090      351.0
2  7D.J57A. -124.450500  47.080101      -55.8
          id   longitude   latitude  elevation
0   UW.MEGW. -123.877090  46.266090      351.0
1   7D.J57A. -124.450500  47.080101      -55.8
2   7D.J49A. -124.427498  46.437801     -120.0
3  7D.FN01A. -124.333702  46.882000      -54.0
          id   longitude   latitude  elevation
0  7D.FN19C. -124.366798  46.730099      -72.0
1   7D.J49C. -124.427803  46.438000     -113.0
2  7D.FN02C. -124.428001  46.949699      -67.0
3  7D.FN01C. -124.333298  46.882301      -56.0
4  7D.FN05C. -124.655602  46.857498     -123.0
5  7D.FN03C. -124.525101  46.887199      -93.0
6   7D.J57C. -124.450600  47.080399      -67.0
7  7D.FN04C. -124.601303  46.917500     -104.0
          id   longitude   latitude  elevation
0  7D.FN03C. -124.525101  46.887199      -93.0
1  7D.FN04C. -124.601303  46.917500     -104.0
2   7D.J49C. -124

In [3]:
stas = pd.concat([stas_2011,stas_2012,stas_2013,
                           stas_2014,stas_2015],ignore_index=True)
stas = stas.drop_duplicates(subset='id')
stas = stas.reset_index(drop=True)
stas

/tmp/ipykernel_3093883/3199595221.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stas = pd.concat([stas_2011,stas_2012,stas_2013,


,id,longitude,latitude,elevation
0,7D.J49A.,-124.427498,46.437801,-120.0
1,UW.MEGW.,-123.877090,46.266090,351.0
2,7D.J57A.,-124.450500,47.080101,-55.8
3,7D.FN01A.,-124.333702,46.882000,-54.0
4,7D.FN19C.,-124.366798,46.730099,-72.0
5,7D.J49C.,-124.427803,46.438000,-113.0
6,7D.FN02C.,-124.428001,46.949699,-67.0
7,7D.FN01C.,-124.333298,46.882301,-56.0
8,7D.FN05C.,-124.655602,46.857498,-123.0
9,7D.FN03C.,-124.525101,46.887199,-93.0


In [4]:
stas.to_csv(f'../data/datasets_{region}/all_stations_{region}.csv')

### 1.2 Concatenate data frames from several stations and format 

In [5]:
df = pd.read_csv('../data/datasets_all_regions/all_picks_all_years_for_assoc.csv',index_col=0)

In [6]:
df['station_id'] = df['station_network_code'] + '.' + df['station_code']+'.'
df = df[df['station_id'].isin(list(stas['id']))]

In [7]:
len(df['station_id'].drop_duplicates())

12

In [8]:
len(df)

2697991

In [9]:
# df.to_csv('../data/datasets_all_years/all_picks_all_years_pnsn_jdf_for_picking.csv')

In [10]:
# # Remove picks with the following stations due to the noise: 'FN05A', 'YOUB' and 'MGB'
# df = df[~df['station_code'].isin(['FN05A', 'YOUB', 'MGB','FN14A','FN07A','J41A','J49A',
# 'J25B','J33B','FN18A','FN08A','M09B','G17B'])]

In [11]:
# # Remove picks with the following stations due to the noise
# df = df[~df['station_code'].isin(['FN05A', 'YOUB', 'MGB',
#                                   'J41A','J33B','M09B','FN01A','FN08A',
#                                   'FN12A','FN18A','J26A','J34A', 'J41A',
#                                   'M02A','FS03B','FS05B','FS08B','G33B',
#                                   'J17B','M10B','M13B','M18B'
#                                  ])]

In [12]:
len(df)

2697991

In [13]:
df.to_csv(f'../data/datasets_{region}/all_picks_all_years_for_assoc_{region}.csv')

### 1.3 Create a CSV file for picks for the association

In [14]:
df = pd.read_csv(f'../data/datasets_{region}/all_picks_all_years_for_assoc_{region}.csv',index_col=0)

In [15]:
# Assign P to P phases in the df
p_phase = df.loc[df['trace_p_arrival'].notna(), 'phase'] = "P"

In [16]:
# Filter the df 
p_stas_picks_phase = df.loc[df['trace_p_arrival'].notna(), ['station_id','trace_p_arrival','phase','pick_id']]
p_stas_picks_phase

,station_id,trace_p_arrival,phase,pick_id
1177,UW.MEGW.,2011-01-01T05:20:58.886000Z,P,1177
1178,UW.MEGW.,2011-01-01T04:45:17.006000Z,P,1178
1179,UW.MEGW.,2011-01-01T03:41:28.386000Z,P,1179
1180,UW.MEGW.,2011-01-01T02:51:12.066000Z,P,1180
1181,UW.MEGW.,2011-01-01T02:39:27.066000Z,P,1181
...,...,...,...,...
15659219,7D.J49C.,2014-06-29T01:41:04.430600Z,P,15659219
15659220,7D.J49C.,2014-06-29T01:43:28.830600Z,P,15659220
15659221,7D.J49C.,2014-06-29T01:44:44.078600Z,P,15659221
15659222,7D.J49C.,2014-06-29T01:45:25.006600Z,P,15659222


In [17]:
# Rename the df
_p_stas_picks_phase = p_stas_picks_phase.rename(columns={"station_id": "station", "trace_p_arrival": "time"})
_p_stas_picks_phase

,station,time,phase,pick_id
1177,UW.MEGW.,2011-01-01T05:20:58.886000Z,P,1177
1178,UW.MEGW.,2011-01-01T04:45:17.006000Z,P,1178
1179,UW.MEGW.,2011-01-01T03:41:28.386000Z,P,1179
1180,UW.MEGW.,2011-01-01T02:51:12.066000Z,P,1180
1181,UW.MEGW.,2011-01-01T02:39:27.066000Z,P,1181
...,...,...,...,...
15659219,7D.J49C.,2014-06-29T01:41:04.430600Z,P,15659219
15659220,7D.J49C.,2014-06-29T01:43:28.830600Z,P,15659220
15659221,7D.J49C.,2014-06-29T01:44:44.078600Z,P,15659221
15659222,7D.J49C.,2014-06-29T01:45:25.006600Z,P,15659222


In [18]:
# Assign S to S phases in the df
s_phase = df.loc[df['trace_s_arrival'].notna(), 'phase'] = "S"# Filter the df 

In [19]:
# Filter the df 
s_stas_picks_phase = df.loc[df['trace_s_arrival'].notna(), ['station_id','trace_s_arrival','phase','pick_id']]
s_stas_picks_phase

,station_id,trace_s_arrival,phase,pick_id
959,UW.MEGW.,2011-01-01T05:52:27.546000Z,S,959
961,UW.MEGW.,2011-01-01T05:54:55.186000Z,S,961
962,UW.MEGW.,2011-01-01T06:20:59.486000Z,S,962
963,UW.MEGW.,2011-01-01T06:44:08.426000Z,S,963
964,UW.MEGW.,2011-01-01T06:54:59.026000Z,S,964
...,...,...,...,...
15659146,7D.J49C.,2014-06-29T01:46:27.662600Z,S,15659146
15659147,7D.J49C.,2014-06-29T01:41:21.198600Z,S,15659147
15659148,7D.J49C.,2014-06-29T01:41:04.478600Z,S,15659148
15659149,7D.J49C.,2014-06-29T01:35:34.526600Z,S,15659149


In [20]:
# Rename the df
_s_stas_picks_phase = s_stas_picks_phase.rename(columns={"station_id": "station", "trace_s_arrival": "time"})
_s_stas_picks_phase

,station,time,phase,pick_id
959,UW.MEGW.,2011-01-01T05:52:27.546000Z,S,959
961,UW.MEGW.,2011-01-01T05:54:55.186000Z,S,961
962,UW.MEGW.,2011-01-01T06:20:59.486000Z,S,962
963,UW.MEGW.,2011-01-01T06:44:08.426000Z,S,963
964,UW.MEGW.,2011-01-01T06:54:59.026000Z,S,964
...,...,...,...,...
15659146,7D.J49C.,2014-06-29T01:46:27.662600Z,S,15659146
15659147,7D.J49C.,2014-06-29T01:41:21.198600Z,S,15659147
15659148,7D.J49C.,2014-06-29T01:41:04.478600Z,S,15659148
15659149,7D.J49C.,2014-06-29T01:35:34.526600Z,S,15659149


In [21]:
# Concatenate the P and S picks
picks = pd.concat(objs = [_p_stas_picks_phase,_s_stas_picks_phase] , axis=0)
picks

,station,time,phase,pick_id
1177,UW.MEGW.,2011-01-01T05:20:58.886000Z,P,1177
1178,UW.MEGW.,2011-01-01T04:45:17.006000Z,P,1178
1179,UW.MEGW.,2011-01-01T03:41:28.386000Z,P,1179
1180,UW.MEGW.,2011-01-01T02:51:12.066000Z,P,1180
1181,UW.MEGW.,2011-01-01T02:39:27.066000Z,P,1181
...,...,...,...,...
15659146,7D.J49C.,2014-06-29T01:46:27.662600Z,S,15659146
15659147,7D.J49C.,2014-06-29T01:41:21.198600Z,S,15659147
15659148,7D.J49C.,2014-06-29T01:41:04.478600Z,S,15659148
15659149,7D.J49C.,2014-06-29T01:35:34.526600Z,S,15659149


In [22]:
# Swap the time and phase columns
picks = picks.iloc[:,[0,2,1,3]]
picks = picks.reset_index(drop=True)
picks

,station,phase,time,pick_id
0,UW.MEGW.,P,2011-01-01T05:20:58.886000Z,1177
1,UW.MEGW.,P,2011-01-01T04:45:17.006000Z,1178
2,UW.MEGW.,P,2011-01-01T03:41:28.386000Z,1179
3,UW.MEGW.,P,2011-01-01T02:51:12.066000Z,1180
4,UW.MEGW.,P,2011-01-01T02:39:27.066000Z,1181
...,...,...,...,...
2697986,7D.J49C.,S,2014-06-29T01:46:27.662600Z,15659146
2697987,7D.J49C.,S,2014-06-29T01:41:21.198600Z,15659147
2697988,7D.J49C.,S,2014-06-29T01:41:04.478600Z,15659148
2697989,7D.J49C.,S,2014-06-29T01:35:34.526600Z,15659149


In [23]:
picks[0:20]

,station,phase,time,pick_id
0,UW.MEGW.,P,2011-01-01T05:20:58.886000Z,1177
1,UW.MEGW.,P,2011-01-01T04:45:17.006000Z,1178
2,UW.MEGW.,P,2011-01-01T03:41:28.386000Z,1179
3,UW.MEGW.,P,2011-01-01T02:51:12.066000Z,1180
4,UW.MEGW.,P,2011-01-01T02:39:27.066000Z,1181
5,UW.MEGW.,P,2011-01-01T01:51:23.686000Z,1182
6,UW.MEGW.,P,2011-01-01T08:20:43.866000Z,1183
7,UW.MEGW.,P,2011-01-01T08:26:49.026000Z,1192
8,UW.MEGW.,P,2011-01-01T08:50:04.046000Z,1193
9,UW.MEGW.,P,2011-01-01T09:19:33.906000Z,1194


In [24]:
# Save the these picks to the data folder
picks.to_csv(f"../data/datasets_{region}/picks_{region}.csv")